In [1]:
import redis
import pymongo
from typing import List, Tuple, Dict

import sys
sys.path.insert(0, "../")

from src.db_drivers.kv_driver.utils import AbstractKVDatabaseConnection, KVDBConnectionConfig, KeyValueDBInstance

/home/dzigen/Desktop/PersonalAI/.pai_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
redis_config = KVDBConnectionConfig(
    host='localhost', port=6380, need_to_clear=False, 
    db_info={'db': 0}, params={'ss_name': 'sorted_node_pairs', 'hs_name': 'node_pairs', 'value_dtype': float})

In [2]:
redis_conn = redis.Redis(host='localhost', port=6380, db=0)

In [ ]:
ss_name = "sorted_node_pairs"
s_name = "node_pairs"
pair_hash = "qwerty"
base_score = 0

In [ ]:
# create
redis_conn.zadd(ss_name, {pair_hash: base_score})
# read
redis_conn.zrangebyscore(ss_name, 0, '+inf', start=0, num=1)
# delete
redis_conn.zrem(ss_name, "Castilla")
# update
redis_conn.zincrby(ss_name, 50, pair_hash)

In [ ]:
# create
redis_conn.zadd(s_name, {'a': 15, 'b': 10, 'c': 20, 'd': 25})
# read
redis_conn.get("username")
# update
redis_conn.set("full_name", "overridee!")
# delete
redis_conn.delete("username")

In [ ]:
# create
redis_conn.hset
# read
redis_conn.hget
# update 
redis_conn.hset
# delete
redis_conn.hdel()

In [13]:
class RedisKVConnector(AbstractKVDatabaseConnection):
    def __init__(self, config: KVDBConnectionConfig):
        self.config = config

    def open_connection(self):
        self.conn = redis.Redis(
            host=self.config.host, port=self.config.port, 
            db=self.config.db_info['db'])

    def is_open(self):
        try:
            self.conn.ping()
            return True
        except (redis.exceptions.ConnectionError, ConnectionRefusedError):
            return False
        
    def close_connection(self):
        self.conn.close()

    def create(self, items: List[KeyValueDBInstance]):
        filtered_items = []
        for item in items:
            item_exists = self.conn.hexists(self.config.params['hs_name'], item.id)
            if not item_exists:
                filtered_items.append(item)

        if len(filtered_items) > 0:
            self.conn.hset(self.config.params['hs_name'], mapping={item.id: str(item.value) for item in filtered_items})
            self.conn.zadd(self.config.params['ss_name'], {item.id: 0 for item in filtered_items})
    

    def read(self, ids: List[str]):
        values = self.conn.hmget(self.config.params['hs_name'], ids)
        formated_values = []
        for val in values:
            if val is None:
                formated_values.append(val)
            else:
                formated_values.append(self.config.params['value_dtype'](val))

        return formated_values

    def update(self, items: List[KeyValueDBInstance]):
        filtered_items = [item for item in items if self.conn.hexists(self.config.params['hs_name'], item.id)]
        
        self.conn.hset(self.config.params['hs_name'], mapping={item.id: str(item.value) for item in filtered_items})
        self.conn.zadd(self.config.params['ss_name'], {item.id: 0 for item in filtered_items})

    def delete(self, ids: List[str]):
        filtered_ids = [id for id in ids if self.conn.hexists(self.config.params['hs_name'], id)]

        if len(filtered_ids) > 0:
            self.conn.hdel(self.config.params['hs_name'], *filtered_ids)
            self.conn.zrem(self.config.params['ss_name'], *filtered_ids)

    def update_item_score(self, mapping: Dict[str, int]) -> None:
        """ Обновляем скоры использования элементов в ордер сете.
        
        :param mapping: 
        :type mapping: Dict[str, int]
        """
        existed_keys = [id for id in list(mapping.keys()) if self.conn.hexists(self.config.params['hs_name'], id)]
        filtered_mapping = {k: mapping[k] for k in existed_keys}

        if len(existed_keys) > 0:
            for k,v in filtered_mapping.items():
                self.conn.zincrby(self.config.params['ss_name'], v, k)

    def delete_rare_items(self, num: int) -> None:
        """Удаляем элементы, к которым было сделано наименьшее количество обращений.

        :param num: Количество элементов, которое нужно удалить.
        :type num: int
        """
        rarest_values = self.conn.zrangebyscore(self.config.params['ss_name'], 0, "+inf", start=0, num=num)
        if len(rarest_values) > 0:
            self.conn.zrem(self.config.params['ss_name'], *rarest_values)
            self.conn.hdel(self.config.params['hs_name'], *rarest_values)

    def count_items(self):
        return self.conn.hlen(self.config.params['hs_name'])

    def item_exist(self, id: str):
        return self.conn.hexists(self.config.params['hs_name'], id)
        
    def clear(self):
        self.conn.delete(self.config.params['hs_name'])
        self.conn.delete(self.config.params['ss_name'])

In [14]:
redis_kv.close_connection()

In [15]:
redis_kv = RedisKVConnector(redis_config)

In [16]:
redis_kv.open_connection()

In [17]:
items = [KeyValueDBInstance(id='123', value=123), KeyValueDBInstance(id='456', value=456)]

In [18]:
redis_kv.create(items)

In [19]:
redis_kv.count_items()

2

In [10]:
redis_kv.clear()

In [75]:
redis_kv.read(['123'])

[None]

In [94]:
items2 = [KeyValueDBInstance(id='123', value=456)]

In [95]:
redis_kv.update(items2)

In [96]:
redis_kv.read(['123'])

[456.0]

In [105]:
redis_kv.delete(['123'])

In [111]:
redis_kv.count_items()

1

In [20]:
redis_kv.update_item_score({'123': 5, '456': 1})

In [21]:
redis_kv.delete_rare_items(num=1)

In [22]:
redis_kv.read(['456', '123'])

[None, 123.0]